In [2]:
import torch
import logging
# Transformer version 4.9.1 - Newer versions may not work.
from transformers import AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # The code assumes you are using GPU. Otherwise, use map_location=torch.device('cpu') in torch.load()
print('Using device:' + str(device))
PRETRAINED_MODEL = 'facebook/bart-base'
SEQ_LENGTH = 600
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

tokenizer.add_special_tokens(
    {'additional_special_tokens': ['<answer>', '<context>']}
)

md2 = torch.load("model_hotpot_supporting_facts_last.pth", map_location=torch.device('cpu')) # Supporting facts
# md2 = torch.load("model_hotpot_full_context_last.pth") # Full context

Using device:cpu


In [3]:
def inference(review_text, model, device):
    encoded_text = tokenizer(
        review_text,
        padding=True,
        max_length=SEQ_LENGTH,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    input_ids = encoded_text['input_ids']
    with torch.no_grad():
        output = model.generate(input_ids)
    decoded_string = tokenizer.decode(output[0], skip_special_tokens=True)
    logging.debug("Decoded string" + decoded_string)
    print(decoded_string)

    return decoded_string


if __name__ == "__main__":
    review_text = "<answer> a fusional language <context> Typologically, Estonian represents a transitional form from an agglutinating language to a fusional language. The canonical word order is SVO (subject–verb–object)."
    inference(review_text, md2, device)


def get_inference(answer, context):
    valuation_text = "<answer> " + answer + " <context> " + context
    return inference(valuation_text, md2, device)

/home/nmc/anaconda3/envs/questiongen/lib/python3.9/site-packages/transformers/generation_utils.py:1838: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Estonian represents a transitional form from what type of language?


In [29]:
review_text = "<answer> Pavani <context> Pavani is the prime minister of the world's biggest democracy.  World's biggest democracy is not china. \
                World's biggest democracy is India. Nirmal lives in Kerala. Nirmal loves ocean. Pavani is a dolphin."
inference(review_text, md2, device)

Who is a dolphin Nirmal or Pavani?


'Who is a dolphin Nirmal or Pavani?'